In [ ]:
%matplotlib inline


Тренируем классификатор
=====================

В этой работе мы воспользуемся знаниями о построении нейронных сетей, использовании оптимизаторов и функций потери.

И коснёмся проблемы, откуда же нам брать данные.

К вопросу о данных
----------------

Если не хотеть сильно усложнить себе работу, если входными данными являются текст, изображения, аудио и видео,
легче всего пользоваться стандартными модулями python, которые загружают данные в матрицу numpy.
После этого можно преобразовать этот массив в ``torch.*Tensor``.

-  Для работы с изображениями существуют пакеты Pillow, OpenCV
-  В том, что касается аудио, есть пакеты scipy и librosa
-  Что касается текста, с ним работают либо при помощи простых загрузчиков Python или Cython, или NLTK и SpaCy

Специально для задач машинного зрения, разработчики *torch* , создали пакет
``torchvision``, который включает загрузчики для большинства известных датасетов, например 
Imagenet, CIFAR10, MNIST, и т.д. и инструментов для трансформации изображений, а именно: 
``torchvision.datasets`` и ``torch.utils.data.DataLoader``.


В этом примере мы используем датасет CIFAR10.
В нём представлены классы: ‘самолёт’, ‘автомобиль’, ‘птица’, ‘кот’, ‘олень’,
‘собака’, ‘лягушка’, ‘лошадь’, ‘корабль’, ‘грузовик’. Изображения CIFAR-10 имеют размер 
size 3x32x32, т.е. это 3-канальные изображения по 32x32 пикселя.


Обучить классификатор изображений
----------------------------

Для этого реализуем следующие шаги:

1. Загрузить и *нормализовать* датасеты CIFAR10, тренировнчный и проверочный, с помощью
   ``torchvision``
2. Задать конволюционную нейронную сеть
3. Определить функцию потерь
4. Натренировать сеть на тренировочных данных
5. Протестировать сеть на данных

### Загрузка и нормализация CIFAR10

Используя ``torchvision``, загрузить CIFAR10 можно так.



In [ ]:
!pip install torch
!pip install import torchvision
import torchvision
import torchvision.transforms as transforms

На выходе torchvision создаёт изображения типа PILImage со значениями в пределах [0, 1].
Мы переводим их в тензоры в нормализованном пространстве [-1, 1].



In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Изобразим несколько тренировочных изображений, просто для показа.



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(4)))

### Определим CNN (конволюционную нейросеть)

Эта сеть умеет работать с изображениями у которых 3 цветовых канала.



In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

### Определим функцию потерь и оптимизатор

Используем перекрёстную энтропию и стохастический градиентный спуск (SGD) с инерцией.



In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

### Обучаем нейронную сеть

Здесь происходит всё самое важное, хотя мы и просто проходимся по итератору который перебирает данные, и подаём входы в нейронную сеть, а потом оптимизируем.



In [ ]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

### Протестировать обучение на тестовых данных

Мы обучали нейронную сеть путём двух проходов по всем тренировочным данным.
Самое время проверить, обучилась ли сеть хотя бы чему-нибудь.

Проверкой будет служить предсказание метки класса которую выдаёт нейронная сеть, и проверкой относительно *ground-truth*. Если предсказание сбылось, добавляем его в список хороших образцов.

Первый шаг. Покажем изображение из тестовой выборки.



In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

Есть. Теперь посмотрим, что нейронная сеть думает об этих объектах:



In [ ]:
outputs = net(images)

Выведем уверенность (энергию) каждого класса.
Чем больше *энергия*, тем увереннее сеть относится 
 к факту принадлежности изображения к конкретному классу.
Так что берём индекс наиболее уверенного прогноза:



In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[j]]
                              for j in range(4)))

Результат обнадёживает.

Посмотрим, как сеть тестируется на всём наборе данных.



In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Всё что больше 10% (принимать случайное решение относительно одного из 10 классов) -- нас вполне устраивает.
Похоже, что сеть чему-то обучилась.

Посмотрим, какие класса предсказываются хорошо, а какие не очень:



In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))